# Module 4 Handon

## Module Preparing

### The Essentail Library

In [2]:
import pandas as pd # do some data
import numpy as np
import multiprocessing as mp
import functools
import itertools
from string import ascii_lowercase

---

## Prop Table Generation

### Coca [Sample](https://www.english-corpora.org/coca/)

In [3]:
COCA = pd.DataFrame(
    [
        ['defeat',21947],
        ['decet',6],
        ['defect',3973],
        ['deft',1240],
        ['defer',2239],
        ['Deeft',0]
    ],
    columns=['word','frequency']
)
COCA_pop = 1e9
COCA['P(w)'] = COCA['frequency']/COCA_pop
COCA['rank'] = COCA['frequency'].rank(ascending=False).astype(int)

COCA.head(6)

,word,frequency,P(w),rank
0,defeat,21947,2.194700e-05,1
1,decet,6,6.000000e-09,5
2,defect,3973,3.973000e-06,2
3,deft,1240,1.240000e-06,4
4,defer,2239,2.239000e-06,3
5,Deeft,0,0.000000e+00,6


### Wikipedia [sample](https://www.english-corpora.org/wiki/)

In [4]:
WIKI = pd.DataFrame(
    [
        ['defeat',121408],
        ['decet',81],
        ['defect',7793],
        ['deft',814],
        ['defer',1416],
        ['Deeft',0]
    ],
    columns=['word','frequency']
)
WIKI_pop = 1.9e9
WIKI['P(w)'] = WIKI['frequency']/WIKI_pop
WIKI['rank'] = WIKI['frequency'].rank(ascending=False).astype(int)

WIKI.head(6)

,word,frequency,P(w),rank
0,defeat,121408,6.389895e-05,1
1,decet,81,4.263158e-08,5
2,defect,7793,4.101579e-06,2
3,deft,814,4.284211e-07,4
4,defer,1416,7.452632e-07,3
5,Deeft,0,0.000000e+00,6


### IULA Spanish-English Technical Corpus [Sample](https://repositori.upf.edu/handle/10230/20052)

In [5]:
IULA = pd.DataFrame(
    [
        ['defeat',11],
        ['decet',0],
        ['defect',180],
        ['deft',0],
        ['defer',11],
        ['Deeft',0]
    ],
    columns=['word','frequency']
)
IULA_pop = 2.1e6
IULA['P(w)'] = IULA['frequency']/IULA_pop
IULA['rank'] = IULA['frequency'].rank(ascending=False).astype(int)

IULA.head(6)

,word,frequency,P(w),rank
0,defeat,11,0.000005,2
1,decet,0,0.000000,5
2,defect,180,0.000086,1
3,deft,0,0.000000,5
4,defer,11,0.000005,2
5,Deeft,0,0.000000,5


as you can see that The COCA and Wikipedia have the same result on `Deeft` but on the IULA we have the different result

It prove that "Corpus does matter"

---

## Update the tables with Norvig and calculate the final prob

### Channel model probability we use the collected list of errors, from [PeterNorvig's collection](http://norvig.com/ngrams/)

#### from [count_1edit.txt](http://norvig.com/ngrams/count_1edit.txt)

In [6]:
norvig = pd.read_csv(
    'http://norvig.com/ngrams/count_1edit.txt',
    sep='\t',
    encoding = "ISO-8859-1",
    header=None
)
norvig.columns = ['term', 'edit']
norvig = norvig.set_index('term')
norvig.head()

,edit
term,
e|i,917
a|e,856
i|e,771
e|a,749
a|i,559


#### from [count_big.txt](http://norvig.com/ngrams/count_big.txt)

In [7]:
norvig_orig = pd.read_csv(
    'http://norvig.com/ngrams/count_big.txt',
    sep='\t',
    encoding = "ISO-8859-1",
    header=None
)
norvig_orig = norvig_orig.dropna()
norvig_orig.columns=['term','freq']
norvig_orig.head()

,term,freq
0,a,21160
1,aah,1
2,aaron,5
3,ab,2
4,aback,3


### P(X|W)

In [8]:
def get_count(c):
    return norvig_orig.apply(lambda x: x.term.count(c) * x.freq, axis=1).sum()

In [9]:
character_set = list(map(''.join, itertools.product(ascii_lowercase, repeat=1))) + list(map(''.join, itertools.product(ascii_lowercase, repeat=2)))

The code below didn't workout for me so I decide to use the old fashion way 

```
with mp.Pool(processes=8) as pool:
    freq_list = pool.map(functools.partial(get_count), character_set)
```

In [11]:
freq_list = []
for i in character_set :
    freq_list.append(get_count(i))

In [12]:
freq_df = pd.DataFrame([character_set, freq_list], index=['char', 'freq']).T
freq_df = freq_df.set_index('char')
COCA['P(x|w)'] = [
    (norvig.loc['e|ea'].values / freq_df.loc['ea'].values)[0],
    (norvig.loc['f|c'].values / freq_df.loc['c'].values)[0],
    (norvig.loc['e|ec'].values / freq_df.loc['ec'].values)[0],
    (norvig.loc['e| '].values / freq_df.loc['e'].values)[0],
    (norvig.loc['t|r'].values / freq_df.loc['r'].values)[0],
    (norvig.loc['fe|ef'].values / freq_df.loc['ef'].values)[0]
]
COCA.head(6)

,word,frequency,P(w),rank,P(x|w)
0,defeat,21947,2.194700e-05,1,0.012834
1,decet,6,6.000000e-09,5,0.000028
2,defect,3973,3.973000e-06,2,0.003167
3,deft,1240,1.240000e-06,4,0.000003
4,defer,2239,2.239000e-06,3,0.000036
5,Deeft,0,0.000000e+00,6,0.003311


### P(x|w)P(w) — Using COCA

In [15]:
COCA['109 P(x|w)P(w)'] = 1e9 * COCA['P(w)'] * COCA['P(x|w)']
COCA.head(6)

,word,frequency,P(w),rank,P(x|w),109 P(x|w)P(w)
0,defeat,21947,2.194700e-05,1,0.012834,281.667621
1,decet,6,6.000000e-09,5,0.000028,0.000166
2,defect,3973,3.973000e-06,2,0.003167,12.582104
3,deft,1240,1.240000e-06,4,0.000003,0.003918
4,defer,2239,2.239000e-06,3,0.000036,0.079565
5,Deeft,0,0.000000e+00,6,0.003311,0.000000


### P(x|w)P(w) — Using IULA

In [17]:
IULA['P(x|w)'] = COCA['P(x|w)']
IULA['109 P(x|w)P(w)'] = 1e9 * IULA['P(w)'] * IULA['P(x|w)']
IULA.head(6)

,word,frequency,P(w),rank,P(x|w),109 P(x|w)P(w)
0,defeat,11,0.000005,2,0.012834,67.225672
1,decet,0,0.000000,5,0.000028,0.000000
2,defect,180,0.000086,1,0.003167,271.448786
3,deft,0,0.000000,5,0.000003,0.000000
4,defer,11,0.000005,2,0.000036,0.186141
5,Deeft,0,0.000000,5,0.003311,0.000000


## Demonstrate the result

In [18]:
# number 3 goes there